In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [2]:
url = r"https://www.goodreads.com/author/quotes/656983.J_R_R_Tolkien?page=1"

driver = webdriver.Chrome()
driver.get(url)

In [58]:
quotes = driver.find_elements(By.CLASS_NAME, "quoteText")
quotes = [quote.text for quote in quotes]

In [59]:
print(quotes[0])

“All that is gold does not glitter,
Not all those who wander are lost;
The old that is strong does not wither,
Deep roots are not reached by the frost.

From the ashes a fire shall be woken,
A light from the shadows shall spring;
Renewed shall be blade that was broken,
The crownless again shall be king.”
― J.R.R. Tolkien, The Fellowship of the Ring


In [60]:
print(quotes[-1])

“Do not meddle in the affairs of wizards, for they are subtle and quick to anger.”
― J.R.R. Tolkien, The Fellowship of the Ring
